This notebook is for downloading your own MAL data, with the overall rating of the show set to the one given in MAL. 

The rest of the values are by default 0 and need to be manually input.

### Loading


In [1]:
from show import *
from series import *
from colors import color
from operator import itemgetter, attrgetter

In [2]:
import pickle
completed = {}

with open('./show_data/completed_shows.pkl', 'rb') as f:
    completed = pickle.load(f)
    
with open('./show_data/series.pkl', 'rb') as f:
    series = pickle.load(f)
    
with open('./show_data/decisions.pkl', 'rb') as f:
    decisions = pickle.load(f)

### Definitions

In [21]:
def input_safe(txt):
    try:
        inp = float(input(txt))
        if inp < 0 or inp > 10:
            return "error"
        else:
            return inp
    except:
        print('Error')

def create_show(name='', tp='TV'):
    if len(name) == 0:
        name = input("Name of Show: ").strip()
    plot = float(input_safe("Story: "))
    characters = float(input_safe("Characters: "))
    art = float(input_safe("Art/Animation: "))
    music = float(input_safe("Sound: "))
    enjoy = float(input_safe("Personal Enjoyment: "))
    return Show(name, plot, characters, art, music, enjoy, tp)

def create_movie(name=''):
    return create_show(name, 'Movie')

def add_show(show):
    old = completed.pop(show.name, None)
    if old:
        show.set_type(old.get_type())
    completed[show.name] = show
    
    for title in series:
        shows = series[title].shows
        if show in shows:
            series[title].addShow(show)
            break

### Make/Update Entries

In [53]:
def entries(shows=[], ans=''):
    while shows or ans == 'yes' or ans == 'y' or len(ans) > 2:
        if shows:
            name = shows.pop(0)
        else:    
            name = ans
        
        name = name.strip()

        if len(name) > 0 :
            if name in completed:
                print(str(completed[name]))
                
            try:
                x = create_show(name) # change to create_movie to add movie
                print('\n' + str(x))
                add_show(x)
            except TypeError as t_e:
                print()
        else:
            if len(name) == 0:
                print('err: no name entered')
                break
        
        if not shows:
            ans = input('\nAdd another? (yes/no/edit): ')
            
ans = input('Enter name of show: ')
# ans = ''
entries(ans=ans)

Enter name of show: Shingeki no Kyojin Season 2   
Shingeki no Kyojin Season 2             8           8           10          10          9           8.8         9           
Story: 8
Characters: 8
Art/Animation: 10
Sound: 10
Personal Enjoyment: 9

Shingeki no Kyojin Season 2             8           8           10          10          9           8.8         9           

Add another? (yes/no/edit): 


In [54]:
s = input('Enter name of series: ')
if s in series:
    for show in series[s].shows:
        print(show)
    print()
    entries(series[s].getShows())
else:
    print('Series not found')

Enter name of series: Shingeki no Kyojin
Shingeki no Kyojin Season 2             8           8           10          10          9           8.8         9           
Shingeki no Kyojin Season 3 Part 2      10          9           10          10          10          9.7         10          
Shingeki no Kyojin                      8           9           10          10          9           9.1         9           
Shingeki no Kyojin Season 3             9           8           10          10          10          9.1         9           
Shingeki no Kyojin: The Final Se...     10          9           7           9           9           8.9         9           

Shingeki no Kyojin Season 2             8           8           10          10          9           8.8         9           
Story: 
Error

Shingeki no Kyojin Season 3 Part 2      10          9           10          10          10          9.7         10          
Story: 
Error

Shingeki no Kyojin                      8           9

In [32]:
sorted(completed), len(completed)

(['86',
  'Akame ga Kill!',
  'Ao no Exorcist',
  'Boku dake ga Inai Machi',
  'Boku no Hero Academia',
  'Boku no Hero Academia 2nd Season',
  'Boku no Hero Academia 3rd Season',
  'Boku no Hero Academia 4th Season',
  'Code Geass: Hangyaku no Lelouch',
  'Code Geass: Hangyaku no Lelouch R2',
  'Darling in the FranXX',
  'Death Note',
  'Enen no Shouboutai',
  'Enen no Shouboutai: Ni no Shou',
  'Fate/Zero',
  'Fate/Zero 2nd Season',
  "Fate/stay night Movie: Heaven's Feel - I. Presage Flower",
  'Fate/stay night: Unlimited Blade Works',
  'Fate/stay night: Unlimited Blade Works 2nd Season',
  'Fruits Basket 1st Season',
  'Fruits Basket 2nd Season',
  'Fruits Basket: The Final',
  'Fullmetal Alchemist: Brotherhood',
  'Given',
  'Haikyuu!!',
  'Haikyuu!! Second Season',
  'Haikyuu!!: Karasuno Koukou vs. Shiratorizawa Gakuen Koukou',
  'Haikyuu!!: To the Top',
  'Haikyuu!!: To the Top 2nd Season',
  'Hunter x Hunter (2011)',
  'Jujutsu Kaisen',
  'Kanojo, Okarishimasu',
  'Kaze ga Tsu

In [34]:
def percentMatch(show, title):
    matches = 0
    common = ''
    thresh = 8
        
    for i in range(len(show)):
        if i >= len(title):
            break
        
        if title[i] == show[i]:
            matches += 1
            common += title[i]
        else:
            break
    
#     if matches > thresh:
#         return matches, 1, common
    
    return matches, matches / max(len(show), len(title)), common.strip()

In [35]:
# series = {}
def fmtString(num, percent, matches, title, name, common):
    return f'({str(num)}){percent*100:.2f}% match: {title} | {name} --> {common}\n'

for show in sorted(completed):
    add = False
    found = False
    
    for title in series:
        if completed[show] in series[title].shows:
            found = True
            break
    
    if not found:
        print(f'Inserting: {color.BOLD}{show}{color.END}')
        for title in series:
            matches, percent, common = percentMatch(show, title)

            if percent > 0.6 and matches > 5:
                add = True
                print(fmtString(1, percent, matches, title, show, common))
                break
            elif percent > 0.45 or matches > 5:
                if (show, title) in decisions:
                    print(fmtString(2, percent, matches, title, show, common))
                    add = decisions[(show, title)]        
                else:
                    choice = input(fmtString(3, percent, matches, title, show, common))
                    if choice == "y" or choice == "":
                        add = True

                    decisions[(show, title)] = add

                    if add:
                        break

        if add:
            shows = set()
            if common != show and common != title:
                s1 = series.pop(show, None)
                s2 = series.pop(title, None)
                if s1 is not None:
                    shows = shows.union(s1.shows)
                if s2 is not None:
                    shows = shows.union(s2.shows)

                series[common] = Series(common, shows)
            series[common].addShow(completed[show])
        else:
            series[show] = Series(show, {completed[show]})

In [36]:
len(series)

44

### Saving

In [52]:
with open('./show_data/completed_shows.pkl', 'wb') as f:
    pickle.dump(completed, f)
    
with open('./show_data/series.pkl', 'wb') as f:
    pickle.dump(series, f)
    
with open('./show_data/decisions.pkl', 'wb') as f:
    pickle.dump(decisions, f)